In [1]:
import treeUtil
import pickle
import pandas as pd

In [2]:
# !python data/loadIBC.py

In [2]:
with open(r'data/ibcData.pkl', 'rb') as f:
    [lib, con, neutral] = pickle.load(f)

In [3]:
print(lib[0].get_words())

Forcing middle-class workers to bear a greater share of the cost of government weakens their support for needed investments and stirs resentment toward those who depend on public services the most .


In [4]:
print(con[0].get_words())

Gore is getting rich from environmentalism , not just by being paid a whopping $ 175,000 per speech but by using political pressure to force government policy in a direction that benefits his business interests .


In [5]:
print(neutral[0].get_words())

In this country , the beneficiaries of Apple 's success are , first , the designers , who have done wonders working with Steve Jobs to produce products that are beautiful and effective .


In [6]:
type(lib[0]) == treeUtil.nodeObj

True

In [7]:
tree = lib[0]
for node in tree:
    if isinstance(node, treeUtil.nodeObj):
        if hasattr(node, 'label'):
            print(node.pos,':', node.label, ':', node.get_words())
        else:
            print(node.pos,':', 'No Label', ':', node.get_words())

ROOT+S : Liberal : Forcing middle-class workers to bear a greater share of the cost of government weakens their support for needed investments and stirs resentment toward those who depend on public services the most .
S+VP : No Label : Forcing middle-class workers to bear a greater share of the cost of government
VP|<NP-S> : No Label : middle-class workers to bear a greater share of the cost of government
NP : No Label : middle-class workers
S+VP : No Label : to bear a greater share of the cost of government
VP : No Label : bear a greater share of the cost of government
NP : No Label : a greater share of the cost of government
NP : No Label : a greater share
NP|<JJR-NN> : No Label : greater share
PP : No Label : of the cost of government
NP : No Label : the cost of government
NP : No Label : the cost
PP : No Label : of government
S|<VP-.> : No Label : weakens their support for needed investments and stirs resentment toward those who depend on public services the most .
VP : Liberal : w

In [8]:
for leaf in tree.get_leaves():
    leaf.print_leaf()

Forcing :  VBG
middle-class :  NN
workers :  NNS
to :  TO
bear :  VB
a :  DT
greater :  JJR
share :  NN
of :  IN
the :  DT
cost :  NN
of :  IN
government :  NP+NN
weakens :  VBZ
their :  PRP$
support :  NN
for :  IN
needed :  JJ
investments :  NNS
and :  CC
stirs :  VBZ
resentment :  NP+NN
toward :  IN
those :  NP+DT
who :  WHNP+WP
depend :  VBP
on :  IN
public :  JJ
services :  NNS
the :  DT
most :  JJS
. :  .


In [9]:
t = lib[0]
# print(len(t))
for node in t:
    if isinstance(node, treeUtil.nodeObj):
        if hasattr(node, 'label'):
            label = node.label
        else:
            label = None
        print(label, ':', node.get_words())
#     print(type(node))
#     if node.parent:
#         print(node.parent.pos)

Liberal : Forcing middle-class workers to bear a greater share of the cost of government weakens their support for needed investments and stirs resentment toward those who depend on public services the most .
None : Forcing middle-class workers to bear a greater share of the cost of government
None : middle-class workers to bear a greater share of the cost of government
None : middle-class workers
None : to bear a greater share of the cost of government
None : bear a greater share of the cost of government
None : a greater share of the cost of government
None : a greater share
None : greater share
None : of the cost of government
None : the cost of government
None : the cost
None : of government
None : weakens their support for needed investments and stirs resentment toward those who depend on public services the most .
Liberal : weakens their support for needed investments and stirs resentment toward those who depend on public services the most
None : weakens their support for needed 

In [10]:
def readNodes(tree, root_id, treeList):
    for node in tree:
        if isinstance(node, treeUtil.nodeObj):
            if hasattr(node, 'label'):
                label = node.label
            else:
                label = None
            is_root = True if 'root' in node.pos.lower() else False
            treeList.append([node.get_words(), label, node.pos, is_root, root_id])
    return treeList

In [11]:
def trees_to_df(trees):
    cols = ['text','label','position','is_root','root_id']
    treeList = []
    for root_id, tree in enumerate(trees):
        readNodes(tree, root_id, treeList)
    df = pd.DataFrame(treeList, columns=cols)
    
    return df

In [12]:
libDf = trees_to_df(lib)
libDf.head()

,text,label,position,is_root,root_id
0,Forcing middle-class workers to bear a greater...,Liberal,ROOT+S,True,0
1,Forcing middle-class workers to bear a greater...,None,S+VP,False,0
2,middle-class workers to bear a greater share o...,None,VP|<NP-S>,False,0
3,middle-class workers,None,NP,False,0
4,to bear a greater share of the cost of government,None,S+VP,False,0


In [13]:
libDf.loc[libDf.label != None].shape

(84345, 5)

In [14]:
conDf = trees_to_df(con)
conDf.head()

,text,label,position,is_root,root_id
0,"Gore is getting rich from environmentalism , n...",Conservative,ROOT+S,True,0
1,"is getting rich from environmentalism , not ju...",None,S|<VP-.>,False,0
2,"is getting rich from environmentalism , not ju...",Conservative,VP,False,0
3,"getting rich from environmentalism , not just ...",Conservative,VP,False,0
4,"getting rich from environmentalism , not just ...",Conservative,VP,False,0


In [15]:
conDf.loc[conDf.label != None].shape

(69252, 5)

In [16]:
neutDf = trees_to_df(neutral)
neutDf.head()

,text,label,position,is_root,root_id
0,"In this country , the beneficiaries of Apple '...",Neutral,ROOT+S,True,0
1,In this country,None,PP,False,0
2,this country,None,NP,False,0
3,", the beneficiaries of Apple 's success are , ...",None,"S|<,-NP-VP-.>",False,0
4,"the beneficiaries of Apple 's success are , fi...",None,S|<NP-VP-.>,False,0


In [17]:
neutDf.loc[neutDf.label != None].shape

(22554, 5)